In [ ]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
import numpy as np
import time

file = pd.read_csv("productSellerIds.csv").to_numpy()
lists = [[],[],[]]

In [ ]:
# Dražen
def getReviewSite(stars, page, ownerID, productID):
    reqParams = {
     'ownerMemberId': str(ownerID),
     'memberType': 'seller',
     'productId': str(productID),
     'companyId': '',
     'evaStarFilterValue': str(stars) + ' Stars',
     'evaSortValue': 'sortdefault@feedback',
     'page': str(page),
     'currentPage': str(page),
     'startValidDate': '',
     'i18n': 'true',
     'withPictures': 'false',
     'withAdditionalFeedback': 'false',
     'onlyFromMyCountry': 'false',
     'version': '',
     'isOpened': 'true',
     'translate': ' Y ',
     'jumpToTop': 'false',
     'v': '2' 
     }
    
    page = req.get("https://feedback.aliexpress.com/display/productEvaluation.htm?", params=reqParams)
    htmlCode = BeautifulSoup(page.content, 'html.parser')
    
    return htmlCode

In [ ]:
# Dražen
def extractComments(html):
    lastSite = False   
    allComments = html.findAll('dt', {'class' :'buyer-feedback'})
    parsedComments = [comment.find('span').text for comment in allComments if comment.find('span').text!=""]
    if(len(allComments)<10):
        lastSite = True     
    return parsedComments, lastSite

In [ ]:
# Antonio
def gather():
  maxCommentsData = 10000
  for i in file:
    if (len(lists[0]) >= maxCommentsData) and (len(lists[1]) >= maxCommentsData) and (len(lists[2]) >= maxCommentsData):
      break
    last5 = False
    last4 = False
    last3 = False
    last2 = False
    last1 = False
    for page in range(1,9):
      if len(lists[0]) < maxCommentsData:
        if not last5:
          passed = False
          while not passed:
            try:
              page5 = getReviewSite(5, page, i[2], i[1])
              passed = True
            except:
              time.sleep(10)
          comments, last5 = extractComments(page5)
          lists[0] += comments
          time.sleep(1) 
        if not last4:
          passed = False
          while not passed:
            try:
              page4 = getReviewSite(4, page, i[2], i[1])
              passed = True
            except:
              time.sleep(10)
          comments, last4 = extractComments(page4)
          lists[0] += comments
          time.sleep(1)
      if len(lists[1]) < maxCommentsData:
        if not last3:
          passed = False
          while not passed:
            try:
              page3 = getReviewSite(3, page, i[2], i[1])
              passed = True
            except:
              time.sleep(10)
          comments, last3 = extractComments(page3)
          lists[1] += comments
          time.sleep(1) 
      if len(lists[2]) < maxCommentsData:
        if not last2:
          passed = False
          while not passed:
            try:
              page2 = getReviewSite(2, page, i[2], i[1])
              passed = True
            except:
              time.sleep(10)
          comments, last2 = extractComments(page2)
          lists[2] += comments 
          time.sleep(1)
        if not last1:
          passed = False
          while not passed:
            try:
              page1 = getReviewSite(1, page, i[2], i[1])
              passed = True
            except:
              time.sleep(10)
          comments, last1 = extractComments(page2)
          lists[2] += comments
          time.sleep(1)
    print(i[0])

In [ ]:
# Antonio
gather()
lenMax = min(10000, len(lists[0]), len(lists[1]), len(lists[2]))
commentsData = np.array([commentsData[0][0:lenMax],commentsData[1][0:lenMax],commentsData[2][0:lenMax]])
df = pd.DataFrame({'positive' : commentsData[0], 'neutral' : commentsData[1], 'negative' : commentsData[2]})
df.to_csv('commentsData.csv')
